In [2]:
!pip install selenium
!pip install bs4
# brew install chromedriver via terminal before continue

In [52]:
from selenium import webdriver
from bs4 import BeautifulSoup 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.maximize_window()

In [53]:
def login():
    try:
        driver.get('https://www.finnomena.com/')
        driver.find_element_by_link_text('ลงชื่อเข้าใช้').click()
        driver.find_element_by_id('username-login').send_keys('')
        driver.find_element_by_id('username-login').send_keys('###EMAIL###')
        driver.find_element_by_class_name('next-process').click()
        driver.find_element_by_id('password-login').send_keys('###PASSWORD###')
        driver.find_element_by_class_name('submit-btn').click()
    except Exception as e:
        print(e)
login()

In [54]:
def getStockDetails(name,isAnnual):
    # stock-growthrate
    sections = ['stock-growthrate','stock-financial-report','stock-financial-ratio','stock-statistics']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    headers = []
    quarters = []
    data = []
    toggled = False
    try:
        for section in sections:
            obj = {}
            keys = []
            names = []
            headerElements = driver.find_element_by_css_selector(f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            if len(headers) == 0:
                for div in BeautifulSoup(headerElements.get_attribute('innerHTML'), 'html.parser').find_all('div', {'class': 'topic'}):
                    headers.append(div.get_text())
            if not toggled and isAnnual:
                toggled = True
                driver.find_element_by_class_name('toggle').click()
                
            for topic in headerElements.find_elements_by_class_name('topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                name = BeautifulSoup(topic.get_attribute('innerHTML'), 'html.parser').get_text()
                keys.append(key)
                names.append(name)
                
                
            contentElements = driver.find_element_by_css_selector(f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]} {names[i]}"
                obj[key] = values
            
            data.append(obj)
        return (headers,quarters,data)
    except Exception as e:
        print(e) 

In [62]:
import pandas as pd
def getFinancialDetails(symbol):
    headers,quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=True)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data['index'] = quarterlyOrAnnual
    df = pd.DataFrame(data) 
    df = df.iloc[:-1]
    df.set_index('index',inplace = True)
    df = df.transpose()
    df.to_csv(f'./Reports/{symbol}.csv')
    return df
getFinancialDetails('ptt')

index,2553,2554,2555,2556,2557,2558,2559,2560,2561,2562
Revenue รายได้รวม,"1,943,858.63","2,475,494.57","2,845,717.79","2,883,905.98","2,866,883.40","2,063,727.44","1,737,144.57","2,044,951.64","2,367,958.54","2,261,039.45"
RevenueYoY การเติบโตต่อปี (%),19.84,27.35,14.96,1.34,-0.59,-28.01,-15.82,17.72,15.80,-4.52
NetProfit กำไรสุทธิ,"83,087.72","105,296.41","104,665.81","94,652.27","55,794.93","19,936.42","94,609.08","135,179.60","119,683.94","92,950.60"
NetProfitYoY การเติบโตต่อปี (%),39.53,26.73,-0.60,-9.57,-41.05,-64.27,374.55,42.88,-11.46,-22.34
EarningPerShare กำไรต่อหุ้น (EPS),29.26,36.91,36.64,33.07,19.33,6.73,32.68,46.74,4.15,3.20
EarningPerShareYoY การเติบโตต่อปี (%),38.94,26.14,-0.73,-9.74,-41.55,-65.18,385.59,43.02,-91.12,-22.89
Asset สินทรัพย์รวม,"1,249,147.52","1,402,412.09","1,631,319.94","1,801,721.57","1,779,179.16","2,173,996.19","2,232,331.08","2,232,314.16","2,355,483.87","2,484,438.68"
TotalDebt หนี้สินรวม,"677,834.94","758,463.52","899,655.89","978,543.37","911,837.04","1,086,308.81","1,060,734.07","983,758.42","1,036,989.52","1,183,399.06"
Equity ส่วนของผู้ถือหุ้น,"490,925.05","555,920.21","605,783.87","682,646.66","697,578.41","697,147.03","762,947.65","818,671.75","875,083.83","878,604.11"
PaidUpCapital มูลค่าหุ้นที่เรียกชำระแล้ว,"28,490.42","28,563.00","28,563.00","28,563.00","28,563.00","28,563.00","28,563.00","28,563.00","28,563.00","28,563.00"
